In [17]:
import pandas as pd
import glob
import os
import numpy as np
import shutil
import sys
import newlinejson
 
# adding Folder_2/subfolder to the system path
sys.path.insert(0, '../src')
 

from helper import *

# Preparing validation data

In [2]:
annotations_step3 = pd.read_csv('../data/processed/annotation_data_step_3.csv')
annotations_step3.drop_duplicates(subset=['title'], inplace=True)
samples = annotations_step3.groupby('corrected_label', group_keys=False).apply(lambda x: x.sample(5))

In [12]:
raw_files = glob.glob('../../data/rawdata_step3/**/*.jpg')

export_path = '../../data/prediction_test/'

for _ in raw_files:
    file_name = os.path.basename(_)[:-4]
    try:
        label = samples[samples['title'] == file_name]['corrected_label'].values[0]
        path = os.path.join(export_path, label)
        if os.path.isdir(path):
            shutil.copy(_, path)
        else:
            os.mkdir(path)
    except:
        pass

In [6]:
export_path = '../../data/prediction_test/'

In [7]:
labels = []
validation_data = glob.glob(export_path + '**/*.jpg')

for _ in validation_data:
    paths = os.path.split(_)
    label = paths[0].split('/')[-1]
    labels.append((paths[1], label))



## make predictions

In [8]:
df = pd.read_csv('../../output_dir/20220906_classification_reportPlaces.csv')

In [9]:
classes = df['Unnamed: 0'][:-3].values
print(f'Classes are: {classes}')

Classes are: ['akker' 'amfitheater' 'aula' 'auto' 'auto_ongeluk' 'bakkerij'
 'basketbal_korfbal' 'begraafplaats' 'begrafenis' 'bibliotheek_boekwinkel'
 'biljart' 'binnen_zwembad' 'bloemen' 'bloementuin' 'boksen'
 'borden_gevelsteen' 'bos_park' 'boten' 'bouwplaats' 'brand' 'brug'
 'bruiloft' 'buiten_zwembad' 'bus_truck' 'cafe' 'carnaval' 'casino'
 'catwalk' 'circus' 'cricket' 'dansende_mensen' 'demonstratie'
 'dieren_overig' 'duinen' 'eend' 'etalage' 'etende_mensen' 'fabriek'
 'fietsende_mensen' 'garage_showroom' 'gebouw' 'geestelijken' 'golf'
 'groepsportret' 'gymnastiek' 'handbal' 'hardlopen' 'haven' 'helikopter'
 'herdenking' 'historisch_toneelstuk' 'hockey' 'hond' 'honkbal' 'huisje'
 'kaartspel' 'kade' 'kamperen' 'kantoor' 'kapper' 'kat' 'kerk_binnen'
 'kerk_buiten' 'kermis' 'kerstmis' 'keuken' 'klaslokaal' 'koe' 'konijn'
 'kunstwerk' 'luchtfoto' 'maquette' 'markt' 'mensen_op_een_boot'
 'mensen_op_trap' 'mensenmassa' 'militair' 'motorfiets' 'muziek_optreden'
 'ongeluk_brancard' 'ont

In [10]:
model_path = '../../output_dir/'
places_model = load_learner(os.path.join(model_path, '20220906_DeBoerPlaces.pkl'))
print('Places model loaded')

Places model loaded


In [11]:
def make_prediction(places_model, classes, img_path, topk=5):
    output_ = {}
    
    # make places predictions
    label, y, output = places_model.predict(img_path)
    output_places = output.numpy()
    output_places = np.expand_dims(output_places, axis=0)

    

    # select top-k
    topk = topk * -1
    
    best_n = np.argsort(output_places, axis=1)[:,topk:]
    probabilities = np.round(np.sort(output_places, axis=1)[:,topk:], 2)
    labels = [classes[i] for i in best_n]
    
    for i, _ in enumerate(probabilities[0]):
        output_[labels[0][i]] = probabilities[0][i]
    #print(output_)
    return output_



In [12]:
results = list()
counter = 1
imgs = glob.glob(export_path + '/**/*.jpg', recursive=True)
print('number of images: {}'.format(len(imgs)))

for img in imgs:
    if counter % 10 == 0:
        print(counter)
    d = dict()
    filename = os.path.basename(img)[:-4]
    correct_label = [x[1] for x in labels if x[0][:-4] == filename][0]
    d['correct_label'] = correct_label
    d['filename'] = filename
    d['predictions'] = make_prediction(places_model, classes, img)
    
    results.append(d)
    counter += 1

number of images: 564


10


20


30


40


...

520


530


540


550


560


In [13]:
top_1 = []
top_5 = []

for x in results:
    if x['correct_label'] in sorted(x['predictions'])[0]:
        top_1.append(1)
        top_5.append(1)
    elif x['correct_label'] in sorted(x['predictions']):
        top_1.append(0)
        top_5.append(1)
    else:
        top_1.append(0)
        top_5.append(0)

In [14]:
print(f'top 1 accuracy: {np.mean(top_1)}')
print(f'top 5 accuracy: {np.mean(top_5)}')

top 1 accuracy: 0.19326241134751773
top 5 accuracy: 0.8634751773049646


In [20]:
filename = 'test_cropped.json'
with open(filename, "w") as f:
    newlinejson.dump(str(results), f)

In [21]:
## TEST RAW IMAGES

In [24]:
raw_images = glob.glob('../../data/resized/**/*.jpg')

In [33]:
imgs_not_found = []
results = list()
counter = 1
imgs = glob.glob('../../data/resized/**/*.jpg', recursive=True)
print('number of images: {}'.format(len(imgs)))

for img in imgs:
    if counter % 10 == 0:
        print(counter)
    d = dict()
    filename = os.path.basename(img)[:-4]
    print(filename)
    try: 
        correct_label = [x[1] for x in labels if x[0][:-4] == filename][0]
        d['correct_label'] = correct_label
        d['filename'] = filename
        d['predictions'] = make_prediction(places_model, classes, img)
    
        results.append(d)
        counter += 1
    except Exception:
        imgs_not_found.append(filename)
        pass


number of images: 706
NL-HlmNHA_1478_05064K00_08


NL-HlmNHA_1478_05389K00_31


NL-HlmNHA_1478_02989K00_28


NL-HlmNHA_1478_02057K00_16


NL-HlmNHA_1478_02986K01_04


...

NL-HlmNHA_1478_45398K01_26


NL-HlmNHA_1478_46557K00_24


NL-HlmNHA_1478_45691K00_04


NL-HlmNHA_1478_45520K00_34


NL-HlmNHA_1478_46349K00_10


In [34]:
top_1 = []
top_5 = []

for x in results:
    if x['correct_label'] in sorted(x['predictions'])[0]:
        top_1.append(1)
        top_5.append(1)
    elif x['correct_label'] in sorted(x['predictions']):
        top_1.append(0)
        top_5.append(1)
    else:
        top_1.append(0)
        top_5.append(0)

In [35]:
print(f'top 1 accuracy: {np.mean(top_1)}')
print(f'top 5 accuracy: {np.mean(top_5)}')

top 1 accuracy: 0.10973451327433628
top 5 accuracy: 0.5451327433628319


In [36]:
filename = 'test_raw.json'
with open(filename, "w") as f:
    newlinejson.dump(str(results), f)